<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install transformers[torch]`
!pip install sacrebleu
!pip install evaluate
!pip install sacrebleu
!pip install accelerate -U

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [7]:
max_length = 256

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
article = "How are you"
inputs = tokenizer(article, return_tensors="pt").to(device)

translated_tokens = model.generate(
     **inputs,  max_length=30
 )
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

In [8]:
#tokenizer.tgt_lang["hin_Deva"]

def preprocess_function(examples):
  #print(examples)
  inputs = [ex["en"] for ex in examples["translation"]]
  targets = [ex["hi"] for ex in examples["translation"]]

  model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)
  labels = tokenizer(targets,max_length=max_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
 # model_inputs = tokenizer(inputs = inputs , text_target= targets, max_length=max_length, truncation=True)
  #model_inputs = tokenizer.prepare_seq2seq_batch(src_texts = inputs , text_target= targets, )
  #print(model_inputs)
  return model_inputs

In [21]:
tokenized_datasets_validation = dataset['validation'].map(
    preprocess_function,
    batched= True,
    remove_columns=dataset["validation"].column_names,
    batch_size = 2
)


In [20]:
tokenized_datasets_test = dataset['test'].map(
    preprocess_function,
    batched= True,
    remove_columns=dataset["test"].column_names,
    batch_size = 2)

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [11]:


# article = "UN Chief says there is no military solution in Syria"
# inputs = tokenizer(article, return_tensors="pt")

# translated_tokens = model.generate(
#     **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["hin_Deva"], max_length=30
# )
# tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

In [12]:

# article = "Accerciser Accessibility Explorer"
# inputs = tokenizer(article, return_tensors="pt")

# translated_tokens = model.generate(
#     **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["hin_Deva"], max_length=30
# )
# tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

In [13]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
import evaluate

metric = evaluate.load("sacrebleu")

import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [23]:

from transformers import Seq2SeqTrainingArguments

model.to(device)
training_args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [27]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=237, training_loss=2.750293071762922, metrics={'train_runtime': 87.2724, 'train_samples_per_second': 86.178, 'train_steps_per_second': 2.716, 'total_flos': 130441967566848.0, 'train_loss': 2.750293071762922, 'epoch': 3.0})

In [ ]:
for i in dataset['test']['translation']:
  print(i['en'])
  break

A black box in your car?


In [26]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 2.9083759784698486,
 'eval_bleu': 0.04461219624339253,
 'eval_runtime': 69.9137,
 'eval_samples_per_second': 7.438,
 'eval_steps_per_second': 0.129,
 'epoch': 3.0}

In [32]:
article = "How are you"
inputs = tokenizer(article, return_tensors="pt").to(device)

translated_tokens = model.generate(
     **inputs,  max_length=30
 )
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

'आप कैसे हैं'